# Article Page Info MediaWiki API Example
This example illustrates how to access page info data using the [MediaWiki REST API for the EN Wikipedia](https://www.mediawiki.org/wiki/API:Main_page). This example shows how to request summary 'page info' for a single article page. The API documentation, [API:Info](https://www.mediawiki.org/wiki/API:Info), covers additional details that may be helpful when trying to use or understand this example.

## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - May 13, 2022



In [5]:
# Third party library installation
!pip install pandas
!pip install tqdm

In [43]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests, os
import pandas as pd
from tqdm import tqdm

The example relies on some constants that help make the code a bit more readable.

In [12]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<amb7896@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# read list of all politician names from provided csv file
politicians_wiki_df = pd.read_csv(os.path.join("..","data","politicians_by_country_SEPT.2022.csv"))
# This is just a list of English Wikipedia article titles that we can use for example requests                            
# ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]
                               
ARTICLE_TITLES = list(politicians_wiki_df.name)        

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [9]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [52]:
politician_current_page_last_revision_id_list = []
for j in tqdm(range(len(ARTICLE_TITLES))):
    politician_article = ARTICLE_TITLES[j]
    # print(f"Getting pageview data for {dino_article} in {mode} mode")
    try:
        info = request_pageinfo_per_article(politician_article)
        # Extracting intermediate pageid key (which is variable for every politician page response) 
        # to get further in to the nested json.
        page_info = info['query']['pages']
        for page in page_info.items():
            page_id = page[0]
        politician_current_page_last_revision_id_list.append(page_info[page_id]["lastrevid"])
    except Exception as e:
        # print names of politician articles who don't have a revision id
        print(politician_article)

 32%|████████████▌                          | 2447/7584 [11:14<21:06,  4.06it/s]

Prince Ofosu Sefah


 39%|███████████████▎                       | 2986/7584 [13:31<19:11,  3.99it/s]

Harjit Kaur Talwandi


 42%|████████████████▌                      | 3213/7584 [14:28<17:46,  4.10it/s]

...

 77%|█████████████████████████████▊         | 5802/7584 [25:27<07:31,  3.95it/s]

Roman Konoplev


 84%|████████████████████████████████▋      | 6345/7584 [29:08<05:45,  3.58it/s]

Nhlanhla “Lux” Dlamini


100%|███████████████████████████████████████| 7584/7584 [38:26<00:00,  3.29it/s]


In [58]:
# Remove politician who's API calls errored out
remove_politician_list = ['Prince Ofosu Sefah','Harjit Kaur Talwandi','Abd al-Razzaq al-Hasani','Kang Sun-nam',
                         'Abiodun Abimbola Orekoya','Segun “Aeroland” Adewale','Roman Konoplev',
                          'Nhlanhla “Lux” Dlamini']
modified_politician_list = list(politicians_wiki_df[~politicians_wiki_df.name.isin(remove_politician_list)].name)


In [59]:
# Create a json with article_name as key and revision_id as value for that politician 
latest_revision_id_dict = {modified_politician_list[i]:politician_current_page_last_revision_id_list[i] 
                           for i in range(len(modified_politician_list))}

with open(os.path.join("..","data","latest_revision_id.json"), "w") as outfile:
    json.dump(latest_revision_id_dict, outfile)

### Switch to wp_ores_example.ipynb for Step 3: Combining the Datasets